In [3]:
import numpy as np
import pandas as pd
import random
from copy import deepcopy
from sklearn.datasets import make_regression

# импортируем мои классы
from TreeReg import MyTreeReg
from LineReg import MyLineReg
from KNNReg import MyKNNReg


In [ ]:
class MyBaggingReg:
    def __init__(self, estimator=None, n_estimators=10, max_samples=1.0, random_state=42, oob_score=None):
        self.estimator = estimator
        self.n_estimators = n_estimators
        self.max_samples = max_samples
        self.random_state = random_state
        self.oob_score = oob_score  # Тип метрики для OOB оценки
        self.oob_score_ = None  # Хранение OOB метрики после обучения
        self.estimators = []  # Список для хранения обученных моделей
        self.oob_predictions = {}  # Словарь для хранения OOB предсказаний по строкам

    def __str__(self):
        params = vars(self) # Получаем все атрибуты экземпляра как словарь
        params_str = ', '.join(f"{key}={value}" for key, value in params.items())
        return f"MyTreeReg class: {params_str}"
    
    def calculate_oob_score(self, y: pd.Series):
        oob_actual = []
        oob_predicted = []

        # Собираем финальные предсказания для строк, у которых есть OOB предсказания
        for idx, preds in self.oob_predictions.items():
            if preds:  # Если для индекса есть предсказания
                oob_actual.append(y.iloc[idx])
                oob_predicted.append(np.mean(preds))  # Среднее предсказание по OOB

        # Вычисляем метрику в зависимости от типа oob_score
        if self.oob_score == 'mae':
            self.oob_score_ = np.mean(np.abs(np.array(oob_actual) - np.array(oob_predicted)))
        elif self.oob_score == 'mse':
            self.oob_score_ = np.mean((np.array(oob_actual) - np.array(oob_predicted)) ** 2)
        elif self.oob_score == 'rmse':
            self.oob_score_ = np.sqrt(np.mean((np.array(oob_actual) - np.array(oob_predicted)) ** 2))
        elif self.oob_score == 'mape':
            self.oob_score_ = np.mean(np.abs((np.array(oob_actual) - np.array(oob_predicted)) / np.array(oob_actual))) * 100
        elif self.oob_score == 'r2':
            ss_total = np.sum((np.array(oob_actual) - np.mean(oob_actual)) ** 2)
            ss_residual = np.sum((np.array(oob_actual) - np.array(oob_predicted)) ** 2)
            self.oob_score_ = 1 - ss_residual / ss_total
    
    def fit(self, X: pd.DataFrame, y: pd.Series):
        # Фиксируем сид для генерации воспроизводимых бутстреп-выборок
        random.seed(self.random_state)

        # Определяем количество строк для бутстреп-выборки
        n_samples = X.shape[0]
        rows_num_list = list(range(n_samples))
        rows_smpl_cnt = int(n_samples * self.max_samples)
        
        self.estimators = []  # Очистка списка моделей
        self.oob_predictions = {i: [] for i in range(n_samples)}  # Подготовка для OOB предсказаний

        for i in range(self.n_estimators):
            # Генерируем случайные индексы строк для бутстреп-выборки
            sample_rows_idx = random.choices(rows_num_list, k=rows_smpl_cnt)
            oob_rows_idx = list(set(rows_num_list) - set(sample_rows_idx))  # OOB выборка

            # Создаем подвыборку
            X_sample = X.iloc[sample_rows_idx]
            y_sample = y.iloc[sample_rows_idx]

            # Клонируем базовый оценщик и обучаем на подвыборке
            model = deepcopy(self.estimator)
            
            # Передаем 'N' в 'fit' для MyTreeReg
            if isinstance(model, MyTreeReg):
                model.fit(X_sample, y_sample, N=X_sample.shape[0])
            else:
                model.fit(X_sample, y_sample)
            
            # Сохраняем обученную модель
            self.estimators.append(model)
            
            # Выполняем предсказания для OOB-выборки и сохраняем их
            if oob_rows_idx:
                X_oob = X.iloc[oob_rows_idx]
                y_oob_pred = model.predict(X_oob)

                for idx, pred in zip(oob_rows_idx, y_oob_pred):
                    self.oob_predictions[idx].append(pred)
                    
        # Рассчитываем OOB метрику по завершении обучения всех моделей
        if self.oob_score:
            self.calculate_oob_score(y)
            
    def predict(self, X: pd.DataFrame):
        # Создаем пустой список для хранения предсказаний для каждой строки
        averaged_predictions = []
        
        # Пройдемся по каждой строке датафрейма X
        for _, row in X.iterrows():
            # Список для предсказаний текущей строки от каждой базовой модели
            row_predictions = []
            
            # Получаем предсказание от каждой модели для текущей строки
            for model in self.estimators:
                prediction = model.predict(pd.DataFrame([row]))[0]  # Предсказание для одной строки
                row_predictions.append(prediction)
            
            # Усредняем предсказания базовых моделей и добавляем в итоговый список
            averaged_predictions.append(sum(row_predictions) / len(row_predictions))
        
        # Возвращаем список предсказаний для всех строк датафрейма
        return averaged_predictions

In [5]:
# Тестирование OOB error
X, y = make_regression(n_samples=100, n_features=5, noise=0.1, random_state=42)
X = pd.DataFrame(X, columns=[f"feature_{i}" for i in range(X.shape[1])])
y = pd.Series(y, name="target")

# Создаем экземпляр MyTreeReg и MyBaggingReg
tree_model = MyTreeReg(max_depth=5, min_samples_split=2, max_leafs=20, bins=5)
bagging_model = MyBaggingReg(estimator=tree_model, n_estimators=10, max_samples=0.8, random_state=42, oob_score='mse')

# Обучаем модель
bagging_model.fit(X, y)

# Выводим OOB оценку
print("OOB MSE:", bagging_model.oob_score_)

OOB MSE: 5446.425240836741


In [ ]:
# Тестировани предсказания
X, y = make_regression(n_samples=100, n_features=5, noise=0.1, random_state=42)
X = pd.DataFrame(X, columns=[f"feature_{i}" for i in range(X.shape[1])])
y = pd.Series(y, name="target")

# Создаем три экземпляра моделей с использованием классов
tree_model = MyTreeReg(max_depth=5, min_samples_split=2, max_leafs=20, bins=5)
line_model = MyLineReg()
knn_model = MyKNNReg()

# Обучаем каждую модель на данных
N = X.shape[0]
tree_model.fit(X, y, N)  # Для дерева передаем N
line_model.fit(X, y) 
knn_model.fit(X, y)   

# Получаем предсказания и считаем сумму для каждой модели отдельно
# Линейная регрессия
line_predictions = line_model.predict(X)
sum_line_predictions = np.sum(line_predictions)
print(f"Сумма для линейной модели: {sum_line_predictions:.2f}")

# KNN
knn_predictions = knn_model.predict(X)
sum_knn_predictions = np.sum(knn_predictions)
print(f"Сумма для KNN модели: {sum_knn_predictions:.2f}")

# Дерево решений
tree_predictions = tree_model.predict(X)
sum_tree_predictions = np.sum(tree_predictions)
print(f"Сумма для модели дерева: {sum_tree_predictions:.2f}")

Сумма для линейной модели: 888.11
Сумма для KNN модели: 958.67
Сумма для модели дерева: 884.07


In [ ]:
# Тестирование обучения для TreeReg
# # Генерация данных для теста
X, y = make_regression(n_samples=100, n_features=5, noise=0.1, random_state=42)
X = pd.DataFrame(X, columns=[f"feature_{i}" for i in range(X.shape[1])])
y = pd.Series(y, name="target")

# Создаем экземпляр MyTreeReg и MyBaggingReg
tree_model = MyTreeReg(max_depth=5, min_samples_split=2, max_leafs=20, bins=5)
bagging_tree = MyBaggingReg(estimator=tree_model, n_estimators=10, max_samples=0.8, random_state=42)

# Обучаем Bagging модель
bagging_tree.fit(X, y)

# Подсчитаем количество листьев для каждой модели в Bagging
leaf_counts = []

for i, model in enumerate(bagging_tree.estimators):
    leaf_counts.append(model.leaf_count())  # Получаем количество листьев для текущей модели

# Теперь вычисляем среднее количество листьев
average_leaf_count = np.mean(leaf_counts)

print(f"\nAverage leaf count for all models: {average_leaf_count:.2f}")



Average leaf count for all models: 18.60


In [13]:
# Тестирование обучения для MyKNNReg
from sklearn.datasets import make_regression

# Генерация данных для теста
X, y = make_regression(n_samples=100, n_features=5, noise=0.1, random_state=42)
X = pd.DataFrame(X, columns=[f"feature_{i}" for i in range(X.shape[1])])
y = pd.Series(y, name="target")

# Создаем экземпляр MyKNNReg и MyBaggingReg
knn_model = MyKNNReg(k=3, metric='euclidean', weight='uniform')
bagging_knn = MyBaggingReg(estimator=knn_model, n_estimators=10, max_samples=0.8, random_state=42)

# Обучаем Bagging модель
bagging_knn.fit(X, y)

# Сумма предсказаний для каждой модели
sum_predictions = []

for i, model in enumerate(bagging_knn.estimators):
    # Получаем предсказания на всех данных
    predictions = model.predict(X)
    # Суммируем предсказания для этой модели
    sum_predictions.append(np.sum(predictions))

# Теперь вычисляем среднее от всех сумм предсказаний
average_sum_predictions = np.mean(sum_predictions)

# Выводим среднее значение суммы предсказаний
print(f"Average of sum of predictions for all models: {average_sum_predictions:.4f}")


Average of sum of predictions for all models: 441.2140


In [9]:
# Тестирование обучения для линейной регрессии
from sklearn.datasets import make_regression

# Генерация данных для теста
X, y = make_regression(n_samples=100, n_features=5, noise=0.1, random_state=42)
X = pd.DataFrame(X, columns=[f"feature_{i}" for i in range(X.shape[1])])
y = pd.Series(y, name="target")

# Создаем экземпляр MyLineReg и MyBaggingReg
linear_model = MyLineReg()
bagging_linear = MyBaggingReg(estimator=linear_model, n_estimators=10, max_samples=0.8, random_state=42)

# Обучаем Bagging модель
bagging_linear.fit(X, y)

# Вывод информации о весах
model = bagging_linear.estimators[0]  # Получаем модель из списка
coef_sum = np.sum(model.weights[1:])  # Используем weights
print(f"(MyLineReg): Sum of weights = {coef_sum:.4f}")


(MyLineReg): Sum of weights = 314.2282


In [5]:
# Тестирование класса
sample1 = MyBaggingReg()
sample2 = MyBaggingReg(n_estimators=100)
sample3 = MyBaggingReg(max_samples=0.7)

# Проверка
print(sample1)
print(sample2)
print(sample3)

MyTreeReg class: estimator=None, n_estimators=10, max_samples=1.0, random_state=42, oob_score=None, oob_score_=None, estimators=[], oob_predictions={}
MyTreeReg class: estimator=None, n_estimators=100, max_samples=1.0, random_state=42, oob_score=None, oob_score_=None, estimators=[], oob_predictions={}
MyTreeReg class: estimator=None, n_estimators=10, max_samples=0.7, random_state=42, oob_score=None, oob_score_=None, estimators=[], oob_predictions={}
